In [30]:
'''
OLS grad => (2/len(yi))*xi.T @((xi @ theta) - yi)
Ridge grad => (2/len(yi))*X.T @ (X @ (theta)-y)+2*lmbda*theta
'''

def learning_schedule(t):
    return t0/(t + t1)

def SGD(xi,yi,theta,eta,lmbda = 0.001):
    #grad = (2/len(yi))*xi.T @((xi @ theta) - yi)
    grad = (2/len(yi))*X.T @ (X @ (theta)-y)+2*lmbda*theta
    theta = theta - eta*grad 
    return theta

def SGDmomentum(xi,yi,theta,eta,momentum=0.9,lmbda=0.001):
    vt = np.zeros((xi.shape[1],yi.shape[1]))
    #gt = 2*xi.T @((xi @ theta) - yi)
    gt = (2/len(yi))*X.T @ (X @ (theta)-y)+2*lmbda*theta
    vt = momentum*vt + eta*gt
    theta = theta - vt
    return theta

def RMSprop(xi,yi,theta,st,eta=1e-3,beta=0.9,eps=1e-8,lmbda = 0.001):
    #gt = 2*xi.T @((xi @ theta) - yi)
    gt = (2/len(yi))*X.T @ (X @ (theta)-y)+2*lmbda*theta
    st = beta*st + (1-beta)*gt*gt
    theta = theta - eta*gt/(np.sqrt(st + eps))
    return theta, st

def ADAM(xi,yi,theta,t,mt,st,eta,beta1=0.90,beta2=0.99,eps=1e-7,lmbda = 0.001):
    #gt = 2*xi.T @((xi @ theta) - yi)
    gt = (2/len(yi))*X.T @ (X @ (theta)-y)+2*lmbda*theta
    mt = beta1*mt + (1-beta1)*gt
    st = beta2*st + (1-beta2)*gt*gt 
    mt = mt/(1-beta1**t)
    st = st/(1-beta2**t)
    theta = theta - eta*mt/(np.sqrt(st)+eps)
    return theta, mt, st

In [49]:
import numpy as np 
from sklearn.utils import resample

# Stochastic Gradient Descent 

n = 300 # num of data points 
x = 2*np.random.randn(m,1)
y = 4+3*x + np.random.randn(m,1)
X = np.zeros((m,2))

X[:,0] = 1 
X[:,1] = x.flatten()

theta = np.random.randn(2,1)



n_epochs = 10000
M = 5 # size of each minibatch 
m = int(n/M) # num of minibatches 

t0, t1 = 5, 50

# initialization
mt = np.zeros((2,1))
st = np.zeros((2,1))
t = 0  

# shuffle the data 
X, y = resample(X,y)
for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X[random_index:random_index+M]
        yi = y[random_index:random_index+M]
        
        eta = learning_schedule(epoch*m + i)
        t = t + 1
        #theta = SGDmomentum(xi,yi,theta,eta)
        #theta = SGD(xi,yi,theta,eta)
        theta, st = RMSprop(xi,yi,theta,st,eta)
        theta, mt, st = ADAM(xi,yi,theta,t,mt,st,eta)

print(theta)

[[4.3568234 ]
 [3.00349114]]
